In [ ]:
"""
Prepare Davison et al. 2023 calving data to use in summer paper

"""

In [ ]:
import xarray as xr
import pandas as pd
import seaborn as sns

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
    
plot_path='/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
file_davison = pd.read_csv(inputpath_rignot + 'Davison_calving.csv',delimiter=';',header=[1,2],index_col=0).dropna(how='all',axis=1)

In [ ]:
file_davison

In [ ]:
file_davison.index.values

In [ ]:
calv_nisf_list = []
for kisf in file_isf.Nisf:
    nname = file_isf['isf_name'].sel(Nisf=kisf).values
    
    if nname == 'Filchner-Ronne':
        davison_row = file_davison.loc['Filchner']
        davison_row.loc['Filchner'] =  file_davison.loc['Filchner'].values + file_davison.loc['Ronne'].values
    elif nname == 'Ross':
        davison_row = file_davison.loc['Ross_East']
        davison_row.loc['Ross_East'] =  file_davison.loc['Ross_East'].values + file_davison.loc['Ross_West'].values
    elif nname == 'Vincennes Bay':
        davison_row  = file_davison.loc['Vincennes_Bay']
    elif nname == 'George VI':
        davison_row  = file_davison.loc['George_VI']
    elif nname == 'Larsen B':
        davison_row  = file_davison.loc['Larsen_B']
    elif nname == 'Larsen C':
        davison_row  = file_davison.loc['Larsen_C']
    elif nname == 'Larsen D':
        davison_row  = file_davison.loc['Larsen_D']
    elif nname == 'Larsen E':
        davison_row  = file_davison.loc['Larsen_E']
    elif nname == 'Larsen F':
        davison_row  = file_davison.loc['Larsen_F']
    elif nname == 'Larsen G':
        davison_row  = file_davison.loc['Larsen_G']
    elif nname == 'Prince Harald':
        davison_row  = file_davison.loc['Prince_Harald']
    elif nname == 'Moscow Univ.':
        davison_row  = file_davison.loc['Moscow_University']
    elif nname == 'Riiser-Larsen':
        davison_row  = file_davison.loc['Riiser_Larsen']
    elif nname == 'Conger/Glenzer':
        davison_row  = file_davison.loc['Conger_Glenzer']
    elif nname == 'Roi Baudouin':
        davison_row  = file_davison.loc['Baudouin']
    elif nname == 'Wilma/Robert/Downer':
        davison_row  = file_davison.loc['Wilma_Robert_Downer']
    elif nname == 'Tracy Tremenchus':
        davison_row  = file_davison.loc['Tracy_Tremenchus']
    elif nname == 'Rayner/Thyer':
        davison_row  = file_davison.loc['Rayner_Thyer']
    elif nname == 'Pine Island':
        davison_row  = file_davison.loc['Pine_Island']
    elif nname == 'Stancomb Brunt':
        davison_row  = file_davison.loc['Brunt_Stancomb']
    elif nname == 'Edward VIII':
        davison_row  = file_davison.loc['Edward_VIII']
    else:
        davison_row  = file_davison.loc[nname]
    
    calv_list = []
    for yy in range(1997,2022):
        Calv_flux = xr.DataArray(data=np.array(davison_row[str(yy)+',5']['observed'].replace(",", ".")).astype(float)).assign_coords({'time': yy})
        calv_list.append(Calv_flux)
        #Calv_flux_unc = xr.DataArray(data=np.array(davison_row['uncertainty']))

    calv_alltime = xr.concat(calv_list, dim='time')    
    calv_nisf_list.append(calv_alltime.assign_coords({'Nisf':kisf}))

calv_all = xr.concat(calv_nisf_list, dim='Nisf')
calv_all.to_dataset(name='calving_flux').to_netcdf(inputpath_davison + 'calving_flux_davison23.nc')

In [ ]:
calv_all